## Demo - use obsession model to predict text

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
import numpy as np

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  #######loading the model from the drive ################################################
  model = torch.load('/content/drive/MyDrive/OCD/model/BioBERT_model_obsession_expert.h5', map_location=torch.device('cuda'))
else:
  print('You can not run the model with cpu, use gpu to use cuda')

You can not run the model with cpu, use gpu to use cuda


In [7]:
def predict_obsession():
  tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
  max_len = 128
  test_input_ids = []
  test_attention_masks = []
  for post in test_posts:
      encoded_dict = tokenizer.encode_plus(
                          post,
                          add_special_tokens = True,
                          truncation = True,
                          max_length = max_len,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',
                    )
      test_input_ids.append(encoded_dict['input_ids'])
      test_attention_masks.append(encoded_dict['attention_mask'])
  test_input_ids = torch.cat(test_input_ids, dim=0)
  test_attention_masks = torch.cat(test_attention_masks, dim=0)
  ###############preparing the data in dataloader###################################################
  batch_size = 16
  test_dataset = TensorDataset(test_input_ids, test_attention_masks)
  test_dataloader = DataLoader(
              test_dataset,  #test data
              sampler = SequentialSampler(test_dataset), #if batches, pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )
  ############ use the model for classification ###################################################
  predictions = []
  for batch in test_dataloader:
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          with torch.no_grad():
              output= model(b_input_ids,
                                    token_type_ids=None,
                                    attention_mask=b_input_mask)
              logits = output.logits
              logits = logits.detach().cpu().numpy()
              pred_flat = np.argmax(logits, axis=1).flatten()

              predictions.extend(list(pred_flat))
  ################################# displaying predicted value ################################
  df_pred = pd.DataFrame()
  df_pred['Input Text'] = df_temp['post']
  df_pred['BioBERT obsession model prediction'] = predictions
  return df_pred

In [8]:
######### Use model for prediction ################################################################
import pandas as pd
######### replace your input text path ############################################################
inputextpath = '/Users/areejnasser/Desktop/ontology-based classification/PhD-2024/Toward Obsessive-Compulsive Disorder Classification System/Classification of OCD_Forum/data/ontology_labeled_data_all_after_Embedding.csv'

df_pred = pd.read_csv(inputextpath)
#df_pred['post'] = df_pred['post'].apply(lambda x:clean_text(x))
df_temp = df_pred.dropna()
df_temp = df_temp[:5] #you can adjust the input any size by changing 2, or provide all by omiting [:2]
test_posts = df_temp['post'].values
result = predict_obsession()
result

/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


NameError: name 'model' is not defined